<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Acc_Dis_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-06-29 11:21:22


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=ea7ed3d89008b6282fe2e17e37834e7371db779ecdaa0a95dce57c352ca30cdf
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [6]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [7]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    #tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

df_prec_dev

,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,Med_PE,Conviction,LatestQtr,StarStock,Criteria,Strategy
1232,2023-06-28 00:00:00+05:30,27400.0,3MINDIA.NS,17.59,1.13,1.70,29373.0,15332.0,14.05,7.0,NaN,NaN,NaN,M,1.0,1.0,V40N,NaN
1232,2023-06-28 00:00:00+05:30,359.0,5PAISA.NS,12.97,0.63,2.09,553.0,96.0,42.45,54.0,347.06,228.0,NaN,M,1.0,1.0,V40N,ATH
1232,2023-06-28 00:00:00+05:30,23278.0,ABBOTINDIA.NS,12.19,1.02,0.10,23278.0,6634.0,0.00,0.0,NaN,NaN,NaN,M,1.0,1.0,V40,NaN
1232,2023-06-28 00:00:00+05:30,2424.0,AKZOINDIA.NS,7.36,1.15,0.62,2508.0,1360.0,7.32,3.0,NaN,NaN,NaN,M,1.0,1.0,V40,NaN
678,2023-06-28 00:00:00+05:30,1703.0,ANGELONE.NS,27.18,1.12,2.93,1889.0,207.0,11.06,11.0,NaN,NaN,NaN,M,1.0,1.0,V40N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,2023-06-28 00:00:00+05:30,725.0,TTKPRESTIG.NS,-10.27,-1.36,0.28,1165.0,391.0,56.85,61.0,NaN,NaN,NaN,M,1.0,0.0,V40N,NaN
880,2023-06-28 00:00:00+05:30,38.0,UJJIVANSFB.NS,36.25,3.57,3.03,57.0,13.0,43.18,50.0,34.87,1350.0,16.8,M,1.0,1.0,V40N,ATH
1232,2023-06-28 00:00:00+05:30,1836.0,VINATIORGA.NS,-6.04,-0.97,-0.69,2342.0,452.0,26.77,28.0,NaN,NaN,NaN,M,0.0,0.0,V40N,NaN
1232,2023-06-28 00:00:00+05:30,609.0,VIPIND.NS,-7.16,0.30,0.32,743.0,200.0,24.68,22.0,NaN,NaN,NaN,M,1.0,0.0,V40N,NaN


In [8]:
cols = ['Date', 'Close', 'Stock', 'Dev%_200', 'StarStock','Criteria']
df_prec_dev = df_prec_dev[cols]

In [9]:
df_acc_dis = pd.DataFrame()

for stock_id in stock_ids:
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      list1 = [stock_id, round(tmp[tmp['Dev%_200']<0]['Dev%_200'].mean(),0), round(tmp[tmp['Dev%_200']>0]['Dev%_200'].mean(),0)]
      tmp1 = pd.DataFrame(list1).T
      tmp1.columns = ['Stock', 'Acc', 'Dis']
      df_acc_dis = df_acc_dis.append(tmp1)

df_acc_dis

,Stock,Acc,Dis
0,3MINDIA.NS,-8.0,8.0
0,5PAISA.NS,-14.0,25.0
0,ABBOTINDIA.NS,-5.0,15.0
0,AKZOINDIA.NS,-6.0,8.0
0,ANGELONE.NS,-8.0,42.0
...,...,...,...
0,TTKPRESTIG.NS,-9.0,14.0
0,UJJIVANSFB.NS,-19.0,23.0
0,VINATIORGA.NS,-6.0,16.0
0,VIPIND.NS,-13.0,16.0


In [10]:
df_recomm = pd.merge(df_prec_dev, df_acc_dis)
df_recomm

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
0,2023-06-28 00:00:00+05:30,27400.0,3MINDIA.NS,17.59,1.0,V40N,-8.0,8.0
1,2023-06-28 00:00:00+05:30,359.0,5PAISA.NS,12.97,1.0,V40N,-14.0,25.0
2,2023-06-28 00:00:00+05:30,23278.0,ABBOTINDIA.NS,12.19,1.0,V40,-5.0,15.0
3,2023-06-28 00:00:00+05:30,2424.0,AKZOINDIA.NS,7.36,1.0,V40,-6.0,8.0
4,2023-06-28 00:00:00+05:30,1703.0,ANGELONE.NS,27.18,1.0,V40N,-8.0,42.0
...,...,...,...,...,...,...,...,...
72,2023-06-28 00:00:00+05:30,725.0,TTKPRESTIG.NS,-10.27,0.0,V40N,-9.0,14.0
73,2023-06-28 00:00:00+05:30,38.0,UJJIVANSFB.NS,36.25,1.0,V40N,-19.0,23.0
74,2023-06-28 00:00:00+05:30,1836.0,VINATIORGA.NS,-6.04,0.0,V40N,-6.0,16.0
75,2023-06-28 00:00:00+05:30,609.0,VIPIND.NS,-7.16,0.0,V40N,-13.0,16.0


In [11]:
df_recomm.to_csv('/content/drive/My Drive/data/stocks/myProspects-Acc-Dis-Signals.csv', index=False)

In [12]:
# Accumulate
df_recomm[df_recomm['Dev%_200'] < df_recomm['Acc']]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
42,2023-06-28 00:00:00+05:30,1038.0,JCHAC.NS,-11.32,0.0,V40N,-11.0,12.0
60,2023-06-28 00:00:00+05:30,542.0,RAJESHEXPO.NS,-18.98,1.0,V40N,-12.0,12.0
72,2023-06-28 00:00:00+05:30,725.0,TTKPRESTIG.NS,-10.27,0.0,V40N,-9.0,14.0
74,2023-06-28 00:00:00+05:30,1836.0,VINATIORGA.NS,-6.04,0.0,V40N,-6.0,16.0


In [13]:
# Disburse
df_recomm[df_recomm['Dev%_200'] > df_recomm['Dis']]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
0,2023-06-28 00:00:00+05:30,27400.0,3MINDIA.NS,17.59,1.0,V40N,-8.0,8.0
8,2023-06-28 00:00:00+05:30,4582.0,BAJAJ-AUTO.NS,19.94,1.0,V40,-6.0,9.0
17,2023-06-28 00:00:00+05:30,7781.0,CERA.NS,28.95,1.0,V40N,-7.0,18.0
18,2023-06-28 00:00:00+05:30,1690.0,COLPAL.NS,9.60,1.0,V40,-4.0,7.0
22,2023-06-28 00:00:00+05:30,88.0,EQUITASBNK.NS,40.16,1.0,V40N,-13.0,21.0
26,2023-06-28 00:00:00+05:30,5104.0,GILLETTE.NS,6.20,0.0,V40,-7.0,5.0
32,2023-06-28 00:00:00+05:30,660.0,HDFCLIFE.NS,20.38,0.0,V40,-9.0,12.0
36,2023-06-28 00:00:00+05:30,1330.0,ICICIGI.NS,15.22,1.0,V40,-7.0,10.0
37,2023-06-28 00:00:00+05:30,574.0,ICICIPRULI.NS,22.18,0.0,V40,-10.0,14.0
38,2023-06-28 00:00:00+05:30,1425.0,INDIGOPNTS.NS,9.87,1.0,V40N,-15.0,5.0


In [14]:
stock_id = 'BAJFINANCE.NS'
df_recomm[df_recomm['Stock'] == stock_id]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
11,2023-06-28 00:00:00+05:30,7070.0,BAJFINANCE.NS,7.8,1.0,V40,-14.0,18.0


In [15]:
df_recomm['Var'] = df_recomm['Dis'] - df_recomm['Acc']
df_recomm[df_recomm['Var']>25].sort_values(by = 'Var', ascending=False)

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis,Var
68,2023-06-28 00:00:00+05:30,7599.0,TATAELXSI.NS,9.76,0.0,V40N,-16.0,38.0,54.0
4,2023-06-28 00:00:00+05:30,1703.0,ANGELONE.NS,27.18,1.0,V40N,-8.0,42.0,50.0
20,2023-06-28 00:00:00+05:30,4375.0,DIXON.NS,19.21,1.0,V40N,-12.0,38.0,50.0
46,2023-06-28 00:00:00+05:30,1498.0,LUXIND.NS,-2.54,0.0,V40N,-19.0,30.0,49.0
73,2023-06-28 00:00:00+05:30,38.0,UJJIVANSFB.NS,36.25,1.0,V40N,-19.0,23.0,42.0
1,2023-06-28 00:00:00+05:30,359.0,5PAISA.NS,12.97,1.0,V40N,-14.0,25.0,39.0
24,2023-06-28 00:00:00+05:30,804.0,FINCABLES.NS,22.67,0.0,V40N,-13.0,24.0,37.0
9,2023-06-28 00:00:00+05:30,1520.0,BAJAJFINSV.NS,1.59,1.0,V40,-15.0,21.0,36.0
40,2023-06-28 00:00:00+05:30,616.0,ISEC.NS,23.93,0.0,V40N,-13.0,23.0,36.0
22,2023-06-28 00:00:00+05:30,88.0,EQUITASBNK.NS,40.16,1.0,V40N,-13.0,21.0,34.0
